In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
df = df = pd.read_csv("/Users/mollyrudisill/Voxcroft/cleandata_processed.csv", index_col="Unnamed: 0")

In [3]:
subset = df[:3000]

In [4]:
def create_distance_matrix(names):
    n = len(names)
    distance_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i != j:
                distance_matrix[i][j] = fuzz.ratio(names[i], names[j])
            else:
                distance_matrix[i][j] = 100  # Self similarity is 100%
    
    return pd.DataFrame(distance_matrix, index=names, columns=names)

# Create distance matrix for the names in the dataset
names = subset['Name'].tolist()
distance_matrix = create_distance_matrix(names)

distance_matrix

,ALLAN BOESAK,RAMAPHOSA,RAMP,RAMP,ALLAN BOESAK,RAMAPHOSA,@MLINDELIMM MKP,@MGJIGGA MKP,@MKYLPARTY MKP,STOCKVILLE ROAD,...,EDWIN SWALES,CORNE’ MULDER,ALEC HOGG,JOHN STEENHUISEN,RAMP,PETER BROWN DRIVE,STEENHUISEN,JOHN STEENHUISEN,DROP'IN BOO KO AIRPORT,JOHN STEENHUISEN
ALLAN BOESAK,100.0,48.0,12.0,12.0,100.0,48.0,30.0,25.0,31.0,37.0,...,33.0,24.0,38.0,29.0,12.0,28.0,17.0,29.0,29.0,29.0
RAMAPHOSA,48.0,100.0,62.0,62.0,48.0,100.0,17.0,29.0,26.0,25.0,...,19.0,18.0,33.0,16.0,62.0,15.0,20.0,16.0,26.0,16.0
RAMP,12.0,62.0,100.0,100.0,12.0,62.0,21.0,38.0,33.0,21.0,...,12.0,24.0,15.0,0.0,100.0,10.0,0.0,0.0,23.0,0.0
RAMP,12.0,62.0,100.0,100.0,12.0,62.0,21.0,38.0,33.0,21.0,...,12.0,24.0,15.0,0.0,100.0,10.0,0.0,0.0,23.0,0.0
ALLAN BOESAK,100.0,48.0,12.0,12.0,100.0,48.0,30.0,25.0,31.0,37.0,...,33.0,24.0,38.0,29.0,12.0,28.0,17.0,29.0,29.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PETER BROWN DRIVE,28.0,15.0,10.0,10.0,28.0,15.0,19.0,7.0,19.0,38.0,...,34.0,33.0,23.0,30.0,10.0,100.0,36.0,30.0,31.0,30.0
STEENHUISEN,17.0,20.0,0.0,0.0,17.0,20.0,15.0,9.0,8.0,31.0,...,35.0,25.0,20.0,81.0,0.0,36.0,100.0,81.0,12.0,81.0
JOHN STEENHUISEN,29.0,16.0,0.0,0.0,29.0,16.0,19.0,14.0,7.0,26.0,...,36.0,34.0,16.0,100.0,0.0,30.0,81.0,100.0,21.0,100.0
DROP'IN BOO KO AIRPORT,29.0,26.0,23.0,23.0,29.0,26.0,27.0,24.0,22.0,27.0,...,29.0,23.0,13.0,21.0,23.0,31.0,12.0,21.0,100.0,21.0


In [5]:
def classify_relationship(score):
    if score >= 80:
        return 2  # Most Likely Related
    elif score >= 60:
        return 1  # Possibly Related
    elif score >= 40:
        return -1  # Less Likely Related
    else:
        return -2  # Not at All Related

# Create distance matrix for the names in the dataset
names = subset['Name'].tolist()
distance_matrix = create_distance_matrix(names)

# Apply classification
relationship_matrix = distance_matrix.applymap(classify_relationship)

# Adding scores to the dataframe as new rows
new_rows = []

for i in range(len(names)):
    for j in range(i + 1, len(names)):
        score = classify_relationship(distance_matrix.iloc[i, j])
        new_row = subset.iloc[i].copy()
        new_row['Name'] = f"{names[i]} & {names[j]}"
        new_row['Relationship_Score'] = score
        new_rows.append(new_row)

# Convert the new rows to a DataFrame and concatenate with the original DataFrame
new_rows_df = pd.DataFrame(new_rows)
df_extended = pd.concat([subset, new_rows_df], ignore_index=True)

/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_2198/1988542645.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  relationship_matrix = distance_matrix.applymap(classify_relationship)


In [6]:
df_extended

,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,Article,Frequency,Relationship_Score
0,6/11/24 4:00,RT @TheStar_news: Dr Allan Boesak warns: DA's ...,ALLAN BOESAK,People,twitter.com,The Star,[Primary: Politics|97% |Secondary: Government ...,"['DA', 'South Africa']",['nan'],"['0', '2']",11.0,2.0,NaN
1,6/11/24 4:00,RT @TheStar_news: Dr Allan Boesak warns: DA's ...,RAMAPHOSA,People,twitter.com,The Star,[Primary: Politics|97% |Secondary: Government ...,"['DA', 'South Africa']",['nan'],['1'],11.0,1.0,NaN
2,6/11/24 3:58,168798: Stationary Vehicle on N3 Eastbound aft...,RAMP,People,twitter.com,i-traffic KZN,"[Primary: Disaster, accident and emergency inc...",['nan'],['Natural Disasters'],"['0', '1']",12.0,2.0,NaN
3,6/11/24 2:23,168798: Stationary Vehicle on N3 Eastbound aft...,RAMP,People,twitter.com,i-traffic KZN,"[Primary: Disaster, accident and emergency inc...",['nan'],['Natural Disasters'],"['0', '1']",18.0,2.0,NaN
4,6/11/24 2:00,RT @TheStar_news: Dr Allan Boesak warns: DA's ...,ALLAN BOESAK,People,twitter.com,The Star,[Primary: Politics|97% |Secondary: Government ...,"['DA', 'South Africa']",['nan'],"['0', '2']",23.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4501495,5/26/24 16:58,DA leader Steenhuisen tells voters 'help is on...,STEENHUISEN & DROP'IN BOO KO AIRPORT,People,twitter.com,SA Breaking News,[Primary: Politics|95% |Secondary: Election|99...,"['ANC', 'DA', 'John Steenhuisen']",['nan'],"['0', '2']",19652.0,2.0,-2.0
4501496,5/26/24 16:58,DA leader Steenhuisen tells voters 'help is on...,STEENHUISEN & JOHN STEENHUISEN,People,twitter.com,SA Breaking News,[Primary: Politics|95% |Secondary: Election|99...,"['ANC', 'DA', 'John Steenhuisen']",['nan'],"['0', '2']",19652.0,2.0,2.0
4501497,5/26/24 16:58,DA leader Steenhuisen tells voters 'help is on...,JOHN STEENHUISEN & DROP'IN BOO KO AIRPORT,People,twitter.com,SA Breaking News,[Primary: Politics|95% |Secondary: Election|99...,"['ANC', 'DA', 'John Steenhuisen']",['nan'],['1'],19652.0,1.0,-2.0
4501498,5/26/24 16:58,DA leader Steenhuisen tells voters 'help is on...,JOHN STEENHUISEN & JOHN STEENHUISEN,People,twitter.com,SA Breaking News,[Primary: Politics|95% |Secondary: Election|99...,"['ANC', 'DA', 'John Steenhuisen']",['nan'],['1'],19652.0,1.0,2.0
